<a href="https://colab.research.google.com/github/gopalm-ai/Quantum_Machine_Learning_An_Applied_Approach/blob/main/kernel_inference_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libraries
! pip3 install pyquil
! pip3 install qutip

In [2]:
from pyquil import get_qc, Program
from pyquil.gates import CNOT, Z, MEASURE
from pyquil.api import local_forest_runtime

In [4]:
! [ -z "$(which qvm)" ] &&\
  apt update &&\
  apt install jq &&\
  export LATEST_FOREST_SDK_VERSION=$(curl -s https://downloads.rigetti.com/qcs-sdk/versions | jq -r '.versions[0].sdk') &&\
  curl -f https://downloads.rigetti.com/qcs-sdk/forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb.tar.bz2 -o $PWD/forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb.tar.bz2 &&\
  tar -xf forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb.tar.bz2 &&\
  ./forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb/forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb.run &&\
  quilc --version &&\
  qvm --version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [5]:
import subprocess

subprocess.Popen(["qvm", "--quiet", "-S"])
subprocess.Popen(["quilc", "--quiet", "-R"])

In [6]:
from pyquil import Program, get_qc
from pyquil.gates import *
import cmath
import matplotlib.pyplot as plt
import numpy as np
from pyquil.api import QCSClientConfiguration, EngagementManager
from pyquil.api import WavefunctionSimulator
from pyquil.api import QVM
from qutip import Bloch
from tempfile import mkdtemp
import itertools
import socket
import subprocess
qvm = QVM()

In [7]:
## define a function for amplitudes

def get_amplitudes(circuit):
  wf_sim = WavefunctionSimulator()
  wavefunction = wf_sim.wavefunction(circuit)
  amplitudes = wavefunction.amplitudes
  return amplitudes

In [21]:
ancilla_qubit = 0
index_qubit = 1
data_qubit = 2
class_qubit = 3
training_set = [[0, 1], [np.sqrt(2)/2, np.sqrt(2)/2]]
labels = [0, 1]
test_set = [[1, 0]]
test_angles = [2*np.arccos(test_set[0][0])/2]
training_angle = (2*np.arccos(training_set[1][0]))/4
angles = [test_angles[0], training_angle]
circuit = Program()

In [22]:
# Create uniform superpositions of the ancilla and index qubits
circuit += H(ancilla_qubit)
circuit += H(index_qubit)

In [23]:
# Entangle the test instance with ground state of ancilla
circuit += CNOT(ancilla_qubit, data_qubit)
circuit += X(ancilla_qubit)

In [24]:
# App;y Identity to Class state
circuit += I(class_qubit)
print('Input = \n', get_amplitudes(circuit))

Input = 
 [0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]


In [25]:
# First training instance
circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += X(index_qubit)
print('First training instance = \n', get_amplitudes(circuit))

First training instance = 
 [0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j 0. +0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]


In [26]:
# Second training instance
circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += CNOT(index_qubit, data_qubit)
circuit += H(data_qubit)

circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += CNOT(index_qubit, data_qubit)
circuit += H(data_qubit)

circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += CNOT(index_qubit, data_qubit)
circuit += H(data_qubit)

circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += CNOT(index_qubit, data_qubit)
circuit += H(data_qubit)

print('Second training instance = \n', get_amplitudes(circuit))

Second training instance = 
 [ 0. +0.j  0. +0.j  0. +0.j  0.5+0.j  0.5+0.j  0.5+0.j -0.5+0.j  0. +0.j
  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]


In [27]:
circuit += CNOT(index_qubit, class_qubit)
#circuit += CNOT(index_qubit, class_qubit)
circuit += H(ancilla_qubit)
print('Circuit \n', get_amplitudes(circuit))

Circuit 
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.70710678+0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.35355339+0.j -0.35355339+0.j
  0.        +0.j  0.        +0.j -0.35355339+0.j -0.35355339+0.j]
